# 신경망(NEURAL NETWORKS)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Net(nn.Module):
    def __init__(self):
        # what is 'super'
        # https://techbard.tistory.com/2930
        super(Net, self).__init__()
        # 1 input image channer, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [3]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [4]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 3, 3])


In [5]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0661,  0.0488,  0.0224,  0.0580,  0.0808,  0.0604, -0.0132, -0.1425,
         -0.0511,  0.0380]], grad_fn=<AddmmBackward>)


In [6]:
net.zero_grad()
out.backward(torch.randn(1, 10))

- nn.Module - 신경망 모듈. 매개변수를 캡슐화(encapsulation) 하는 간편한 방법으로, GPU로 이동, 내보내기(exporting), 불러오기(loading) 등의 작업을 위한 헬퍼(helper)를 제공한다.

## 손실 함수(Loss Function)

In [22]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.7987, grad_fn=<MseLossBackward>)


DAG
```python
input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
         -> view -> linear -> relu -> linear -> relu -> linear
         -> MSELoss
         -> loss
```

In [23]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  #  Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

## 역전파(Backprop)

오차(error)를 역전파하기 위해서는 `loss.backward()`만 해주면 된다. 하지만 기존 변화도를 없애는 작업이 필요한데, 그렇지 않으면 변화도가 기존의 것에 누적되기 때문이다.

In [24]:
net.zero_grad()  # zeroes the gradient buffers of all parameters

print('conv1.bias.gard before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.gard before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0154, -0.0091, -0.0053,  0.0007, -0.0069, -0.0062])


## 가중치 갱신

In [25]:
learning_rate = 0.001
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [26]:
import torch.optim as optim

# Optimizer를 생성
optimizer = optim.SGD(net.parameters(), lr=0.01)

# 학습 과정(training loop)는 다음과 같다
optimizer.zero_grad()  # zero the gradient buffers
outpu = net(input)
loss = criterion(output, target)
optimizer.step()  # Does the update

`optimizer.zero_grad()`를 사용하여 수동으로 변화도 버퍼를 0으로 설정하는 것을 잊으면 안된다. 이는 역전파(Backprop) 섹션에서 설명한 것처럼 변화도가 누적되기 때문이다.